In [1]:
import csv
import os

def write_csv(data, filename):
    # Check if the file already exists
    file_exists = os.path.exists(filename)
    
    # Open the file in append mode
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        
        # Write the header only if the file doesn't exist
        if not file_exists:
            writer.writerow(["Time", 
                             "Day of the week",
                             "CarCount",
                             "BikeCount",
                             "BusCount",
                             "TruckCount",
                             "Total"])
        
        # Write the data
        writer.writerows(data)


In [2]:
import os
import torch
from torchvision import models, transforms
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm  # Import tqdm for Jupyter notebooks
from datetime import datetime, timedelta

# Load a pre-trained Faster R-CNN model from torchvision
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Correct COCO categories for vehicles (based on COCO class IDs)
vehicle_classes = {
    3: "Car",          # COCO ID for Car
    4: "Motorcycle",   # COCO ID for Motorcycle
    6: "Bus",          # COCO ID for Bus
    8: "Truck"         # COCO ID for Truck
}

# Function to process a directory of images
def process_image_directory(directory_path, output_path, threshold=0.5):
    # Get a list of image files in the directory
    image_files = [f for f in os.listdir(directory_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

    # Initialize the transform to convert image to tensor
    transform = transforms.Compose([transforms.ToTensor()])

    # Loop through each image in the directory with a tqdm progress bar
    for image_num, image_file in tqdm(enumerate(image_files), total=len(image_files), desc="Processing images"):
        image_path = os.path.join(directory_path, image_file)
        
        # Load the image
        img = Image.open(image_path)

        # Transform image to tensor for input to the model
        img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

        # Run the image through the model
        with torch.no_grad():
            prediction = model(img_tensor)

        # Extract prediction details
        boxes = prediction[0]['boxes']  # Bounding boxes for detected objects
        labels = prediction[0]['labels']  # Labels for detected objects
        scores = prediction[0]['scores']  # Confidence scores for detected objects

        # Filter out weak detections
        valid_boxes = boxes[scores > threshold]
        valid_labels = labels[scores > threshold]
        valid_scores = scores[scores > threshold]

        # Create a drawing context for the image
        draw = ImageDraw.Draw(img)

        # Loop through the detected objects and draw bounding boxes
        for i, box in enumerate(valid_boxes):
            xmin, ymin, xmax, ymax = box
            draw.rectangle([xmin, ymin, xmax, ymax], outline="red", width=3)

            label = valid_labels[i].item()
            score = valid_scores[i].item()

            # Check if the detected object is a vehicle
            if label in vehicle_classes:
                text = f'{vehicle_classes[label]}: {score:.2f}'
                # Draw the label text
                draw.text((xmin, ymin), text, fill="yellow")

        # Save the modified image with bounding boxes
        img.save(os.path.join(output_path, image_file))
        
        # time in military time
        # Define the start time as a string
        video_start_time = "17:00:00"

        # Convert the start time to a datetime object
        start_time = datetime.strptime(video_start_time, "%H:%M:%S")

        # Calculate the time increment in seconds (image_num * 15 seconds)
        increment = image_num * 15  # Assuming image_num is defined and represents the image index

        # Add the time increment to the start time
        new_time = start_time + timedelta(seconds=increment)

        # Convert the new time back to a string in the same format
        new_time_str = new_time.strftime("%H%M%S")
        
        # number of cars detected
        car_count = len([label for label in valid_labels if label == 3])
        # number of trucks detected
        truck_count = len([label for label in valid_labels if label == 8])
        # number of buses detected
        bus_count = len([label for label in valid_labels if label == 6])
        # number of motorcycles detected
        motorcycle_count = len([label for label in valid_labels if label == 4])
        
        # day of the week
        day_of_week = "Tuesday"
        
        # total number of vehicles detected
        total = car_count + truck_count + bus_count + motorcycle_count
        
        data = [[new_time_str, day_of_week, car_count, motorcycle_count, bus_count, truck_count, total]]
        write_csv(data, "predictions.csv")
        
        

# Example usage
directory_path = 'video_frames_every_15'  # Replace with your directory path
output_path = 'predicted_images'
# create the output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

process_image_directory(directory_path, output_path)


/home/brownjordan317/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/brownjordan317/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing images:   0%|          | 0/137 [00:00<?, ?it/s]